## Knowledge Base
The first thing we need for an agent using RAG is somewhere we want to pull knowledge from. We will use v2 of the AI ArXiv dataset, available on Hugging Face Datasets at [`jamescalam/ai-arxiv2-chunks`](https://huggingface.co/datasets/jamescalam/ai-arxiv2-chunks).

_Note: we're using the prechunked dataset. For the raw version see [`jamescalam/ai-arxiv2`](https://huggingface.co/datasets/jamescalam/ai-arxiv2)._

In [2]:
from datasets import load_dataset

dataset = load_dataset("jamescalam/ai-arxiv2-chunks", split="train[:20000]")
dataset

Dataset({
    features: ['doi', 'chunk-id', 'chunk', 'id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 'primary_category', 'published', 'updated', 'references'],
    num_rows: 20000
})

In [3]:
# this is what the dataset looks like
dataset[1]

{'doi': '2401.09350',
 'chunk-id': 1,
 'chunk': 'These neural networks and their training algorithms may be complex, and the scope of their impact broad and wide, but nonetheless they are simply functions in a high-dimensional space. A trained neural network takes a vector as input, crunches and transforms it in various ways, and produces another vector, often in some other space. An image may thereby be turned into a vector, a song into a sequence of vectors, and a social network as a structured collection of vectors. It seems as though much of human knowledge, or at least what is expressed as text, audio, image, and video, has a vector representation in one form or another.\nIt should be noted that representing data as vectors is not unique to neural networks and deep learning. In fact, long before learnt vector representations of pieces of dataâ\x80\x94what is commonly known as â\x80\x9cembeddingsâ\x80\x9dâ\x80\x94came along, data was often encoded as hand-crafted feature vectors. E

To build our knowledge base we need _two things_:

1. Embeddings, for this we will use `VoyageEmbeddings` using Voyage AI's embedding models, which do need an [API key](https://dash.voyageai.com/api-keys).
2. A vector database, where we store our embeddings and query them. We use Pinecone which again requires a [free API key](https://app.pinecone.io).

First we initialize our connection to Voyage AI and define an `embed` object for embeddings, then we initialize our connection to Pinecone:

In [4]:
from dotenv import load_dotenv
import os
from pinecone import Pinecone
from langchain_community.embeddings import VoyageEmbeddings
# from getpass import getpass (not using this)
load_dotenv()
voyage_key = os.getenv("VOYAGE_API_KEY") #or getpass("Voyage API key: ")
pc_key = os.getenv("PINECONE_API_KEY") #or getpass("Pinecone API key: ")

# Load the embedding model
embed = VoyageEmbeddings(
    voyage_api_key=voyage_key, model="voyage-2"
)

voyage_key = os.getenv("VOYAGE_API_KEY") #or getpass("Voyage API key: ")
pc_key = os.getenv("PINECONE_API_KEY") #or getpass("Pinecone API key: ")

# configure client
pc = Pinecone(api_key=pc_key)

Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [5]:
from pinecone import PodSpec # Use free tier

spec = PodSpec(
    environment="gcp-starter"
)

Before creating an index, we need the dimensionality of our Voyage AI embedding model, which we can find easily by creating an embedding and checking the length:

In [6]:
vec = embed.embed_documents(["ello"])
len(vec[0])

1024

### Creating the index
Now we create the index using our embedding dimensionality, and a metric also compatible with the model (this can be either cosine or dotproduct). We also pass our spec to index initialization.

In [9]:
import time

index_name = "claude-3-rag"

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=len(vec[0]),  # dimensionality of voyage model
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.2,
 'namespaces': {'': {'vector_count': 20000}},
 'total_vector_count': 20000}

### Populating our Index

Now our knowledge base is ready to be populated with our data. We will use the `embed` helper function to embed our documents and then add them to our index.

We will also include metadata from each record.

In [7]:
dataset.to_pandas().head()

,doi,chunk-id,chunk,id,title,summary,source,authors,categories,comment,journal_ref,primary_category,published,updated,references
0,2401.09350,0,4 2 0 2\nn a J 7 1 ] S D . s c [\n1 v 0 5 3 9 ...,2401.09350#0,Foundations of Vector Retrieval,Vectors are universal mathematical objects tha...,http://arxiv.org/pdf/2401.09350,Sebastian Bruch,"cs.DS, cs.IR",None,None,cs.DS,20240117,20240117,[]
1,2401.09350,1,These neural networks and their training algor...,2401.09350#1,Foundations of Vector Retrieval,Vectors are universal mathematical objects tha...,http://arxiv.org/pdf/2401.09350,Sebastian Bruch,"cs.DS, cs.IR",None,None,cs.DS,20240117,20240117,[]
2,2401.09350,2,If new and old knowledge can be squeezed into ...,2401.09350#2,Foundations of Vector Retrieval,Vectors are universal mathematical objects tha...,http://arxiv.org/pdf/2401.09350,Sebastian Bruch,"cs.DS, cs.IR",None,None,cs.DS,20240117,20240117,[]
3,2401.09350,3,"Similarity is then a function of two vectors, ...",2401.09350#3,Foundations of Vector Retrieval,Vectors are universal mathematical objects tha...,http://arxiv.org/pdf/2401.09350,Sebastian Bruch,"cs.DS, cs.IR",None,None,cs.DS,20240117,20240117,[]
4,2401.09350,4,A neural network that is trained to perform a ...,2401.09350#4,Foundations of Vector Retrieval,Vectors are universal mathematical objects tha...,http://arxiv.org/pdf/2401.09350,Sebastian Bruch,"cs.DS, cs.IR",None,None,cs.DS,20240117,20240117,[]


In [ ]:
from tqdm.auto import tqdm

# easier to work with dataset as pandas dataframe
data = dataset.to_pandas()

batch_size = 100

for i in tqdm(range(0, len(data), batch_size)):
    i_end = min(len(data), i+batch_size)
    # get batch of data
    batch = data.iloc[i:i_end]
    # generate unique ids for each chunk
    ids = [f"{x['doi']}-{x['chunk-id']}" for i, x in batch.iterrows()]
    # get text to embed
    texts = [x['chunk'] for _, x in batch.iterrows()]
    # embed text
    embeds = embed.embed_documents(texts)
    # get metadata to store in Pinecone
    metadata = [
        {'text': x['chunk'],
         'source': x['source'],
         'title': x['title']} for i, x in batch.iterrows()
    ]
    # add to Pinecone
    index.upsert(vectors=zip(ids, embeds, metadata))

  0%|          | 0/200 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [10]:
# Check index is correctly populated
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.2,
 'namespaces': {'': {'vector_count': 20000}},
 'total_vector_count': 20000}

## Agent

### Creating Tool

In [12]:
from langchain.agents import tool

@tool
def arxiv_search(query: str) -> str:
    """Use this tool when answering questions about AI, machine learning, data
    science, or other technical questions that may be answered using arXiv
    papers.
    """
    # create query vector
    xq = embed.embed_query(query)
    # perform search
    out = index.query(vector=xq, top_k=5, include_metadata=True)
    # reformat results into string
    results_str = "\n\n".join(
        [x["metadata"]["text"] for x in out["matches"]]
    )
    return results_str

tools = [arxiv_search]

When this tool is used by our agent it will execute it like so:

In [14]:
print(
    arxiv_search.run(tool_input={"query": "can you tell me about llama 2?"})
)

Model Llama 2 Code Llama Code Llama - Python Size FIM LCFT Python CPP Java PHP TypeScript C# Bash Average 7B â 13B â 34B â 70B â 7B â 7B â 7B â 7B â 13B â 13B â 13B â 13B â 34B â 34B â 7B â 7B â 13B â 13B â 34B â 34B â â â â â 14.3% 6.8% 10.8% 9.9% 19.9% 13.7% 15.8% 13.0% 24.2% 23.6% 22.2% 19.9% 27.3% 30.4% 31.6% 34.2% 12.6% 13.2% 21.4% 15.1% 6.3% 3.2% 8.3% 9.5% 3.2% 12.6% 17.1% 3.8% 18.9% 25.9% 8.9% 24.8% â â â â â â â â â â 37.3% 31.1% 36.1% 30.4% 29.2% 29.8% 38.0%

Ethical Considerations and Limitations (Section 5.2) Llama 2 is a new technology that carries risks with use. Testing conducted to date has been in English, and has not covered, nor could it cover all scenarios. For these reasons, as with all LLMs, Llama 2âs potential outputs cannot be predicted in advance, and the model may in some instances produce inaccurate or objectionable responses to user prompts. Therefore, before deploying any applications of L

### Defining Agent

To create an agent we need a `prompt`, `llm`, and a list of `tools`. We can download a prebuilt prompt for a conversational agents from LangChain hub.

In [23]:
from langchain import hub 

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlacehold

In [24]:
prompt.dict()

{'name': None,
 'input_variables': ['agent_scratchpad', 'input'],
 'input_types': {'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]],
  'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]},
 'output_parser': None,
 'partial_variables': {},
 'metadata': {'lc_hub_owner': 'hwchase17',
  'lc_hub_repo': 'openai-tools-agent',
  'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'},
 'tags': None,
 'messages': [{'prompt': {'name': None,
    'input_variables

In [21]:
from langchain_openai.chat_models import ChatOpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") #or getpass("OpenAI API key: ")

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo", temperature=0)

In [26]:
## Initialise agent 
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)

In [27]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'}, messages

With our `agent` object initialized we pass it to an `AgentExecutor` object alongside our original `tools` list:

In [28]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)

Now we can use the agent via the `invoke` method:

In [31]:
user_msg = "can you tell me about llama 2?"

out = agent_executor.invoke({
    "input": user_msg
})

print(out["output"])



> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}